In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

Загрузим датасет MNIST и рассмотрим его формат

In [2]:
(xTrain, yTrain), (xTest, yTest) = mnist.load_data()

print(xTrain.shape, yTrain.shape)
print(xTest.shape, yTest.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


Как видно, формат датасета (обучающей выборки) следующий: <br>
<ul>
    <li>60000 образцов изображений</li>
    <li>28 пикселей в высоту</li>
    <li>28 пикселей в ширину</li>
</ul>
То есть это трёхмерный массив данных. Такой формат не подходтит для обучения нейросетей, требуется изименить форму датасета на двухмерный массив. Число образцов оставим неизменным, а двумерный массив пикселей преобразуем в одномерный: 

In [3]:
xTrain = tf.reshape(xTrain, (60000, 28 * 28))
print(xTrain.shape)
xTest = tf.reshape(xTest, (10000, 28 * 28))
print(xTest.shape)

(60000, 784)
(10000, 784)


Рассмотрим формат одного элемента массива:

In [4]:
print(xTrain[0].dtype)
print(xTrain[0][200:250])

<dtype: 'uint8'>
tf.Tensor(
[  0   0   0  49 238 253 253 253 253 253 253 253 253 251  93  82  82  56
  39   0   0   0   0   0   0   0   0   0   0   0   0  18 219 253 253 253
 253 253 198 182 247 241   0   0   0   0   0   0   0   0], shape=(50,), dtype=uint8)


Формат элемента является 8 битным беззнаковым целым, диапазон значений которого лежит в промежутке от 0 до 255. Такой формат для обучения нейросестей не самый удачный. Произведём масштабирование и преобразуем диапазон от 0 до 1. При этом нужно учесть, что сначала нужно преобразовать тип данных в float32

In [5]:
xTrain = tf.cast(xTrain, dtype = tf.float32) / 255
xTest = tf.cast(xTest, dtype = tf.float32) / 255
print(xTrain[0].dtype)
print(xTrain[0][200:250])


<dtype: 'float32'>
tf.Tensor(
[0.         0.         0.         0.19215687 0.93333334 0.99215686
 0.99215686 0.99215686 0.99215686 0.99215686 0.99215686 0.99215686
 0.99215686 0.9843137  0.3647059  0.32156864 0.32156864 0.21960784
 0.15294118 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.07058824 0.85882354 0.99215686 0.99215686 0.99215686
 0.99215686 0.99215686 0.7764706  0.7137255  0.96862745 0.94509804
 0.         0.         0.         0.         0.         0.
 0.         0.        ], shape=(50,), dtype=float32)


Датасет преобразован и готов для обучения моделей <br>
Начнём с обучения логистической регрессии

In [6]:
x = keras.Input((784), dtype = tf.float32)
W = tf.Variable(tf.zeros((784, 10)))
b = tf.Variable(tf.zeros((10)), dtype = tf.float32)
y_true = tf.Variable((10), dtype = tf.float32)
y_pred = tf.matmul(x, W) + b

model = keras.Model(x, y_pred)

The following Variables were used a Lambda layer's call (tf.linalg.matmul), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(784, 10) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.__operators__.add), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(10,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


In [7]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.SGD(learning_rate = 0.5),
    metrics = ['accuracy']
)

In [8]:
model.fit(xTrain, yTrain, batch_size=32, epochs=5, verbose=2)
model.evaluate(xTest, yTest, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 1s - loss: 2.3026 - accuracy: 0.0987
Epoch 2/5
1875/1875 - 1s - loss: 2.3026 - accuracy: 0.0987
Epoch 3/5
1875/1875 - 1s - loss: 2.3026 - accuracy: 0.0987
Epoch 4/5
1875/1875 - 1s - loss: 2.3026 - accuracy: 0.0987
Epoch 5/5
1875/1875 - 1s - loss: 2.3026 - accuracy: 0.0987
313/313 - 0s - loss: 2.3026 - accuracy: 0.0980


[2.30259108543396, 0.09799999743700027]

Точность получилась минимальной, так как данный тип распознавания не является задачей регрессии

<h4>Вторая часть</h4>
Теперь обучим нейронную сеть следующими параметрами:
<ul>
    <li>Входной слой:   784 нейрона</li>
    <li>Скрытый слой 1: 200 нейронов</li>
    <li>Скрытый слой 2: 100 нейронов</li>
    <li>Скрытый слой 3: 60 нейронов</li>
    <li>Скрытый слой 4: 30 нейронов</li>
    <li>Выходной слой:  10 нейронов</li>
</ul>    
Сначала расмотрим упрощённый вариант на основе Sequential API (слои идут друг за другом)

In [3]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(28*28)),
        layers.Dense(200, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(30, activation='relu'),
        layers.Dense(10, activation='relu')
    ]
)

#relu - линейный выпрямитель, rectified linear unit 

 Установим модели функцию потерь и оптимизатор (способ минимизации функции потери)

In [10]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.SGD(learning_rate = 0.5),
    metrics = ['accuracy']
)

#Функция потерь - Кроссэнтропия
#SGD - уменьшает расстояние между полученным значением функции потерь и тем, 
#что должно быть, SGD - Stochastic Gradient Descent

Обучим модель со следующими параметрами:<br>
Размер батча (порция данных): 32 образца<br>
Число эпох: 5

In [11]:
model.fit(xTrain, yTrain, batch_size=32, epochs=5, verbose=2)
model.evaluate(xTest, yTest, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 3s - loss: 1.1872 - accuracy: 0.6211
Epoch 2/5
1875/1875 - 3s - loss: 0.8181 - accuracy: 0.6801
Epoch 3/5
1875/1875 - 3s - loss: 0.7698 - accuracy: 0.6870
Epoch 4/5
1875/1875 - 2s - loss: 0.4322 - accuracy: 0.8390
Epoch 5/5
1875/1875 - 3s - loss: 0.1233 - accuracy: 0.9688
313/313 - 0s - loss: 0.1420 - accuracy: 0.9647


[0.14200787246227264, 0.9646999835968018]

Теперь повторим тоже самое, но уже с применением Functional API 
<br>Считается более гибкой

In [12]:
inpLayer = keras.Input(shape=(28*28))
hidLayer1 = layers.Dense(200, activation='relu')(inpLayer)
hidLayer2 = layers.Dense(100, activation='relu')(hidLayer1)
hidLayer3 = layers.Dense(60, activation='relu')(hidLayer2)
hidLayer4 = layers.Dense(30, activation='relu')(hidLayer3)
outLayer = layers.Dense(10, activation='relu')(hidLayer4)

model = keras.Model(inputs=inpLayer, outputs=outLayer)

In [13]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.SGD(learning_rate = 0.5),
    metrics = ['accuracy']
)

In [14]:
model.fit(xTrain, yTrain, batch_size=32, epochs=5, verbose=2)
model.evaluate(xTest, yTest, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 3s - loss: 1.0274 - accuracy: 0.6917
Epoch 2/5
1875/1875 - 3s - loss: 0.5885 - accuracy: 0.8673
Epoch 3/5
1875/1875 - 2s - loss: 0.4486 - accuracy: 0.9216
Epoch 4/5
1875/1875 - 3s - loss: 0.3398 - accuracy: 0.9669
Epoch 5/5
1875/1875 - 2s - loss: 0.3201 - accuracy: 0.9725
313/313 - 0s - loss: 0.3248 - accuracy: 0.9707


[0.3247833549976349, 0.9707000255584717]

Точность составила 97%